In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# specify our batch size, number of epochs, and learning rate
batch_size = 1
EPOCHS = 10
LR = 1e-3

In [53]:
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torchvision.transforms
from torch.utils.data import random_split
import torch


batch_size=1

root = 'drive/MyDrive/ZPD_training'

transform = torchvision.transforms.Compose([
            torchvision.transforms.ToTensor(),
            torchvision.transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))])

dataset = ImageFolder(root, transform=transform)
dataset_size = len(dataset)

train_size = int(dataset_size * 0.8)
validate_size = dataset_size - train_size
train_set, val_set = random_split(dataset, [train_size, validate_size], generator=torch.Generator().manual_seed(42))

train_loader = DataLoader(train_set, batch_size = batch_size, num_workers = 1, shuffle = True, pin_memory = True)
val_loader = DataLoader(val_set, batch_size = batch_size, num_workers = 1, shuffle = True, pin_memory = True)

In [ ]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=False)
# or any of these variants
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet34', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet101', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet152', pretrained=True)
model.eval()

In [ ]:
# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

In [ ]:
model = model.to(device)


loss_function = torch.nn.MSELoss()
#loss_function = SSIM()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

for epoch in range(0, EPOCHS):
    # set the model in training mode
    model.train()
    # initialize the total training and validation loss
    totalTrainLoss = 0
    totalValLoss = 0
    # loop over the training set
    for (x, y) in train_loader:
        # send the input to the device
        x = x.float()
        y = y.float()
        (x, y) = (x.to(device), y.to(device))

        # perform a forward pass and calculate the training loss
        pred = model(x)
        loss = loss_function(pred, y)
        # zero out the gradients, perform the backpropagation step,
        # and update the weights
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # add the loss to the total training loss so far and
        # calculate the number of correct predictions
        totalTrainLoss += loss.item()

    with torch.no_grad():
        # set the model in evaluation mode
        model.eval()
        # loop over the validation set
        for (x, y) in val_loader:
            # send the input to the device
            x = x.float()
            y = y.float()
            (x, y) = (x.to(device), y.to(device))
            # make the predictions and calculate the validation loss
            pred = model(x)
            totalValLoss += loss_function(pred, y)


    print(f"EPOCH: {epoch + 1}/{EPOCHS}")
    print(f"Training loss for epoch {epoch + 1}: {totalTrainLoss / (len(train_set))}")
    print(f"Validation Loss for epoch {epoch + 1}: {totalValLoss / (len(val_set))}")
    if totalValLoss / (len(val_set)) < lowest:
        lowest = totalValLoss / (len(val_set))
        print(f"\nNew lowest val loss: {totalValLoss / (len(val_set))}\n")
        torch.save(model.state_dict(), "results/Resnet.pth")
    print("-------------------------------------------------------------")